In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import joblib
import numpy as np


In [2]:
data=pd.read_csv("karnataka_power_consumption2.csv")

In [3]:
data.head(2)

,City,State,Month,Weather Condition,Temperature (°C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,UPS Power Consumed (kWh),Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Supply Status
0,Bangalore,Karnataka,January,Sunny,15.290211,66.547887,71.719208,7.941387,2.886671,1.800727,...,1.352620,0.435386,7.154724,0.821981,0.653049,2.557426,20.251767,0.843824,1.043001,Overflow
1,Mysore,Karnataka,January,Sunny,32.047148,68.040769,4.828414,21.345099,2.583581,2.064290,...,1.222503,0.792027,3.272261,0.788361,4.480270,1.455194,21.423678,0.892653,0.936460,Sufficient


In [4]:
df=data
df

,City,State,Month,Weather Condition,Temperature (°C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,UPS Power Consumed (kWh),Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Supply Status
0,Bangalore,Karnataka,January,Sunny,15.290211,66.547887,71.719208,7.941387,2.886671,1.800727,...,1.352620,0.435386,7.154724,0.821981,0.653049,2.557426,20.251767,0.843824,1.043001,Overflow
1,Mysore,Karnataka,January,Sunny,32.047148,68.040769,4.828414,21.345099,2.583581,2.064290,...,1.222503,0.792027,3.272261,0.788361,4.480270,1.455194,21.423678,0.892653,0.936460,Sufficient
2,Mangalore,Karnataka,January,Sunny,30.343414,93.875090,44.304381,20.342969,0.184108,9.034373,...,0.355890,0.993113,0.045122,0.829571,2.167786,2.476600,20.072195,0.836341,1.184043,Overflow
3,Hubli,Karnataka,January,Sunny,36.828643,59.924131,31.584487,8.533994,3.259899,5.075506,...,0.776721,0.993833,5.095593,0.141463,2.529214,2.241601,24.350766,1.014615,1.302403,Overflow
4,Dharwad,Karnataka,January,Sunny,36.913369,67.443776,57.989981,2.814506,3.899499,9.184095,...,0.334667,0.780912,7.661298,1.200281,3.182193,2.059317,30.173085,1.257212,1.055827,Insufficient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,Chitradurga,Karnataka,December,Cool,34.090821,42.779135,66.531790,18.106481,4.279386,7.735572,...,0.040747,0.089268,6.813243,4.215430,1.257289,2.841925,28.551207,1.189634,1.557060,Overflow
368,Davanagere,Karnataka,December,Cloudy,39.664174,41.619173,36.570164,7.724879,1.150621,5.877105,...,1.471823,0.219055,6.593460,1.174884,4.019927,2.124660,24.946871,1.039453,1.484345,Overflow
369,Ballari,Karnataka,December,Sunny,29.967194,58.708944,48.083400,1.177558,3.287795,8.444214,...,0.002554,0.551664,4.995723,3.190869,1.795689,0.610808,25.792155,1.074673,0.911810,Insufficient
370,Kalaburagi,Karnataka,December,Sunny,22.523888,57.241919,3.126721,16.395308,0.645974,3.093202,...,0.949626,0.430955,0.387552,4.475065,1.074764,1.739277,17.491466,0.728811,1.019782,Overflow


In [5]:
df.isna().sum()

City                                    0
State                                   0
Month                                   0
Weather Condition                       0
Temperature (°C)                        0
Humidity (%)                            0
Rainfall Chances (%)                    0
Wind Speed (km/h)                       0
Fan Power Consumed (kWh)                0
Light Power Consumed (kWh)              0
Mixer Power Consumed (kWh)              0
Washing Machine Power Consumed (kWh)    0
Phone Charging Power Consumed (kWh)     0
UPS Power Consumed (kWh)                0
Grinder Power Consumed (kWh)            0
AC Power Consumed (kWh)                 0
Heater Power Consumed (kWh)             0
Fridge Power Consumed (kWh)             0
TV Power Consumed (kWh)                 0
Total Power Consumed (kWh)              0
Required Power Supply (kW)              0
Current Power Supply (kW)               0
Power Supply Status                     0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 23 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   City                                  372 non-null    object 
 1   State                                 372 non-null    object 
 2   Month                                 372 non-null    object 
 3   Weather Condition                     372 non-null    object 
 4   Temperature (°C)                      372 non-null    float64
 5   Humidity (%)                          372 non-null    float64
 6   Rainfall Chances (%)                  372 non-null    float64
 7   Wind Speed (km/h)                     372 non-null    float64
 8   Fan Power Consumed (kWh)              372 non-null    float64
 9   Light Power Consumed (kWh)            372 non-null    float64
 10  Mixer Power Consumed (kWh)            372 non-null    float64
 11  Washing Machine Pow

In [7]:
cols_encoder=['City','State','Month','Weather Condition','Power Supply Status']

In [8]:
encoders_dict = {}  # For mapping original to encoded values
label_encoders = {}  # For storing LabelEncoder instances

# Encode each column
for col in cols_encoder:
    # Convert values to lowercase for consistency
    df[col] = df[col].str.lower()
    
    # Initialize LabelEncoder
    le = LabelEncoder()
    
    # Fit and transform the column
    df[col] = le.fit_transform(df[col])
    
    # Store mappings in encoders_dict
    encoders_dict[col] = dict(zip(le.classes_, range(len(le.classes_))))
    
    # Store LabelEncoder instance
    label_encoders[col] = le



In [9]:
encoders_dict


{'City': {'bagalkot': 0,
  'ballari': 1,
  'bangalore': 2,
  'belgaum': 3,
  'bidar': 4,
  'bijapur': 5,
  'chikkaballapura': 6,
  'chikkamagaluru': 7,
  'chikmagalur': 8,
  'chitradurga': 9,
  'dakshina kannada': 10,
  'davanagere': 11,
  'dharwad': 12,
  'gadag': 13,
  'hassan': 14,
  'haveri': 15,
  'hubli': 16,
  'kalaburagi': 17,
  'kodagu': 18,
  'kolar': 19,
  'koppal': 20,
  'mandya': 21,
  'mangalore': 22,
  'mysore': 23,
  'raichur': 24,
  'ramanagara': 25,
  'shivamogga': 26,
  'tumkur': 27,
  'udupi': 28,
  'yadgir': 29},
 'State': {'karnataka': 0},
 'Month': {'april': 0,
  'august': 1,
  'december': 2,
  'february': 3,
  'january': 4,
  'july': 5,
  'june': 6,
  'march': 7,
  'may': 8,
  'november': 9,
  'october': 10,
  'september': 11},
 'Weather Condition': {'cloudy': 0,
  'cold': 1,
  'cool': 2,
  'hot': 3,
  'rainy': 4,
  'sunny': 5,
  'warm': 6},
 'Power Supply Status': {'insufficient': 0, 'overflow': 1, 'sufficient': 2}}

In [10]:
df.head(5)

,City,State,Month,Weather Condition,Temperature (°C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,UPS Power Consumed (kWh),Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Supply Status
0,2,0,4,5,15.290211,66.547887,71.719208,7.941387,2.886671,1.800727,...,1.352620,0.435386,7.154724,0.821981,0.653049,2.557426,20.251767,0.843824,1.043001,1
1,23,0,4,5,32.047148,68.040769,4.828414,21.345099,2.583581,2.064290,...,1.222503,0.792027,3.272261,0.788361,4.480270,1.455194,21.423678,0.892653,0.936460,2
2,22,0,4,5,30.343414,93.875090,44.304381,20.342969,0.184108,9.034373,...,0.355890,0.993113,0.045122,0.829571,2.167786,2.476600,20.072195,0.836341,1.184043,1
3,16,0,4,5,36.828643,59.924131,31.584487,8.533994,3.259899,5.075506,...,0.776721,0.993833,5.095593,0.141463,2.529214,2.241601,24.350766,1.014615,1.302403,1
4,12,0,4,5,36.913369,67.443776,57.989981,2.814506,3.899499,9.184095,...,0.334667,0.780912,7.661298,1.200281,3.182193,2.059317,30.173085,1.257212,1.055827,0


In [11]:
df.tail(5)

,City,State,Month,Weather Condition,Temperature (°C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,UPS Power Consumed (kWh),Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Supply Status
367,9,0,2,2,34.090821,42.779135,66.531790,18.106481,4.279386,7.735572,...,0.040747,0.089268,6.813243,4.215430,1.257289,2.841925,28.551207,1.189634,1.557060,1
368,11,0,2,0,39.664174,41.619173,36.570164,7.724879,1.150621,5.877105,...,1.471823,0.219055,6.593460,1.174884,4.019927,2.124660,24.946871,1.039453,1.484345,1
369,1,0,2,5,29.967194,58.708944,48.083400,1.177558,3.287795,8.444214,...,0.002554,0.551664,4.995723,3.190869,1.795689,0.610808,25.792155,1.074673,0.911810,0
370,17,0,2,5,22.523888,57.241919,3.126721,16.395308,0.645974,3.093202,...,0.949626,0.430955,0.387552,4.475065,1.074764,1.739277,17.491466,0.728811,1.019782,1
371,7,0,2,2,16.011461,55.093360,80.048832,10.161915,3.737347,3.273593,...,0.866666,0.180535,4.882608,1.609399,2.145777,0.927092,20.191070,0.841295,1.015526,1


In [12]:
df.columns

Index(['City', 'State', 'Month', 'Weather Condition', 'Temperature (°C)',
       'Humidity (%)', 'Rainfall Chances (%)', 'Wind Speed (km/h)',
       'Fan Power Consumed (kWh)', 'Light Power Consumed (kWh)',
       'Mixer Power Consumed (kWh)', 'Washing Machine Power Consumed (kWh)',
       'Phone Charging Power Consumed (kWh)', 'UPS Power Consumed (kWh)',
       'Grinder Power Consumed (kWh)', 'AC Power Consumed (kWh)',
       'Heater Power Consumed (kWh)', 'Fridge Power Consumed (kWh)',
       'TV Power Consumed (kWh)', 'Total Power Consumed (kWh)',
       'Required Power Supply (kW)', 'Current Power Supply (kW)',
       'Power Supply Status'],
      dtype='object')

In [13]:
input_columns = ['City', 'State', 'Month', 'Weather Condition', 'Temperature (°C)',
       'Humidity (%)', 'Rainfall Chances (%)', 'Wind Speed (km/h)',
       'Fan Power Consumed (kWh)', 'Light Power Consumed (kWh)',
       'Mixer Power Consumed (kWh)', 'Washing Machine Power Consumed (kWh)',
       'Phone Charging Power Consumed (kWh)', 'UPS Power Consumed (kWh)',
       'Grinder Power Consumed (kWh)', 'AC Power Consumed (kWh)',
       'Heater Power Consumed (kWh)', 'Fridge Power Consumed (kWh)',
       'TV Power Consumed (kWh)']
target_columns = ['Total Power Consumed (kWh)',
       'Required Power Supply (kW)', 'Current Power Supply (kW)',
       'Power Supply Status']


In [14]:
#avg_features = df.groupby(["City",'State', 'Month']).mean().reset_index()

In [15]:
#avg_features

In [16]:
#avg_features.to_csv("processed_features.csv", index=False)

In [19]:
X=df[input_columns].values
y=df[target_columns].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()


In [21]:
df

,City,State,Month,Weather Condition,Temperature (°C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,UPS Power Consumed (kWh),Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Supply Status
0,2,0,4,5,15.290211,66.547887,71.719208,7.941387,2.886671,1.800727,...,1.352620,0.435386,7.154724,0.821981,0.653049,2.557426,20.251767,0.843824,1.043001,1
1,23,0,4,5,32.047148,68.040769,4.828414,21.345099,2.583581,2.064290,...,1.222503,0.792027,3.272261,0.788361,4.480270,1.455194,21.423678,0.892653,0.936460,2
2,22,0,4,5,30.343414,93.875090,44.304381,20.342969,0.184108,9.034373,...,0.355890,0.993113,0.045122,0.829571,2.167786,2.476600,20.072195,0.836341,1.184043,1
3,16,0,4,5,36.828643,59.924131,31.584487,8.533994,3.259899,5.075506,...,0.776721,0.993833,5.095593,0.141463,2.529214,2.241601,24.350766,1.014615,1.302403,1
4,12,0,4,5,36.913369,67.443776,57.989981,2.814506,3.899499,9.184095,...,0.334667,0.780912,7.661298,1.200281,3.182193,2.059317,30.173085,1.257212,1.055827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,9,0,2,2,34.090821,42.779135,66.531790,18.106481,4.279386,7.735572,...,0.040747,0.089268,6.813243,4.215430,1.257289,2.841925,28.551207,1.189634,1.557060,1
368,11,0,2,0,39.664174,41.619173,36.570164,7.724879,1.150621,5.877105,...,1.471823,0.219055,6.593460,1.174884,4.019927,2.124660,24.946871,1.039453,1.484345,1
369,1,0,2,5,29.967194,58.708944,48.083400,1.177558,3.287795,8.444214,...,0.002554,0.551664,4.995723,3.190869,1.795689,0.610808,25.792155,1.074673,0.911810,0
370,17,0,2,5,22.523888,57.241919,3.126721,16.395308,0.645974,3.093202,...,0.949626,0.430955,0.387552,4.475065,1.074764,1.739277,17.491466,0.728811,1.019782,1


In [22]:
print(X.shape)
print(y.shape)

(372, 19)
(372, 4)


In [23]:

X_train = scaler_X.fit_transform(X_train)
X_val = scaler_X.transform(X_val)
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)

In [24]:
joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [25]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [26]:
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [27]:
import torch.nn.functional as F

In [28]:
class PowerConsumptionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(PowerConsumptionModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256), 
            nn.ELU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(negative_slope=0.02),
            nn.Linear(64,output_size ),
            nn.Softplus()

        )

    def forward(self, x):
        return self.network(x)

In [29]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = PowerConsumptionModel(input_size, output_size)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [30]:
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

epochs = 100
patience = 10
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    val_mae = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            val_loss += loss.item()

            mae = torch.mean(torch.abs(predictions - y_batch))
            val_mae += mae.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    val_mae /= len(val_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Step the scheduler based on the validation loss
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth") 
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

model.load_state_dict(torch.load("best_model.pth"))
print("Best model loaded.")

C:\Users\dhany\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.5818, Val Loss: 0.6189
Epoch 2/100, Train Loss: 0.5608, Val Loss: 0.5992
Epoch 3/100, Train Loss: 0.5378, Val Loss: 0.5805
Epoch 4/100, Train Loss: 0.5320, Val Loss: 0.5635
Epoch 5/100, Train Loss: 0.5270, Val Loss: 0.5491
Epoch 6/100, Train Loss: 0.4907, Val Loss: 0.5344
Epoch 7/100, Train Loss: 0.4947, Val Loss: 0.5227
Epoch 8/100, Train Loss: 0.4738, Val Loss: 0.5100
Epoch 9/100, Train Loss: 0.4637, Val Loss: 0.4970
Epoch 10/100, Train Loss: 0.4755, Val Loss: 0.4889
Epoch 11/100, Train Loss: 0.4361, Val Loss: 0.4769
Epoch 12/100, Train Loss: 0.4170, Val Loss: 0.4619
Epoch 13/100, Train Loss: 0.4142, Val Loss: 0.4538
Epoch 14/100, Train Loss: 0.4141, Val Loss: 0.4489
Epoch 15/100, Train Loss: 0.4127, Val Loss: 0.4429
Epoch 16/100, Train Loss: 0.3856, Val Loss: 0.4342
Epoch 17/100, Train Loss: 0.3974, Val Loss: 0.4283
Epoch 18/100, Train Loss: 0.3783, Val Loss: 0.4218
Epoch 19/100, Train Loss: 0.3896, Val Loss: 0.4196
Epoch 20/100, Train Loss: 0.3735, Val Lo

C:\Users\dhany\AppData\Local\Temp\ipykernel_7212\89324024.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


In [31]:


# Define the model architecture (same as training)
model = PowerConsumptionModel(input_size, output_size)  # Replace with your actual model class
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Set to evaluation mode


C:\Users\dhany\AppData\Local\Temp\ipykernel_7212\4041461524.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


PowerConsumptionModel(
  (network): Sequential(
    (0): Linear(in_features=19, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.02)
    (8): Linear(in_features=64, out_features=4, bias=True)
    (9): Softplus(beta=1.0, threshold=20.0)
  )
)

In [36]:


# Define the model architecture (same as training)
model = PowerConsumptionModel(input_size, output_size)  # Replace with your actual model class
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Set to evaluation mode


C:\Users\dhany\AppData\Local\Temp\ipykernel_7212\4041461524.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


PowerConsumptionModel(
  (network): Sequential(
    (0): Linear(in_features=19, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.02)
    (8): Linear(in_features=64, out_features=4, bias=True)
    (9): Softplus(beta=1.0, threshold=20.0)
  )
)

In [51]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler

num_districts = 31
num_features = 19
num_outputs = 4

# Assuming scaler_X and scaler_y are already fitted
# Assuming model is already trained

# Generate random input data for 31 districts
random_data = np.random.uniform(low=[5, 5, 0, 0, 20, 20, 30, 2, 0.5, 0.05, 1.0, 5.0, 0.01, 2.5, 0.5, 5.0, 5.0, 3.0, 2.0],
                                 high=[10, 10, 1, 1, 35, 35, 50, 10, 0.7, 0.2, 1.5, 8.0, 0.05, 4.0, 1.2, 9.0, 9.0, 5.0, 4.0],
                                 size=(num_districts, num_features))

# Transform input using scaler
tensor_input = scaler_X.transform(random_data)
tensor_input = torch.tensor(tensor_input, dtype=torch.float32)

with torch.no_grad():
    output = model(tensor_input)

# Inverse transform the predicted output
predicted_output = scaler_y.inverse_transform(output.numpy())

# Adjust second and third column relationships randomly
for i in range(num_districts):
    condition = np.random.choice(["greater", "less", "equal"])
    if condition == "greater":
        if predicted_output[i, 1] <= predicted_output[i, 2]:
            predicted_output[i, 1] += 0.1
            predicted_output[i, 2] -= 0.1
    elif condition == "less":
        if predicted_output[i, 1] >= predicted_output[i, 2]:
            predicted_output[i, 1] -= 0.1
            predicted_output[i, 2] += 0.1
    else:  # "equal"
        predicted_output[i, 1] = (predicted_output[i, 1] + predicted_output[i, 2]) / 2
        predicted_output[i, 2] = predicted_output[i, 1]
    
    print(f"District {i+1}: Predicted Outputs:", predicted_output[i])


District 1: Predicted Outputs: [27.480536   1.0080981  1.1650366  0.6983337]
District 2: Predicted Outputs: [26.39703    1.0738317  1.0688807  0.7323875]
District 3: Predicted Outputs: [26.83367    1.0924077  1.0924077  0.7399649]
District 4: Predicted Outputs: [30.49015     1.1740048   1.1740048   0.70523274]
District 5: Predicted Outputs: [30.539892   1.2302512  1.0557103  0.664801 ]
District 6: Predicted Outputs: [29.60204    1.1150134  1.1982114  0.6813484]
District 7: Predicted Outputs: [26.615496    1.176884    0.9900392   0.73016095]
District 8: Predicted Outputs: [27.483423    1.0089129   1.1799841   0.72470284]
District 9: Predicted Outputs: [27.673874   1.0240188  1.1503261  0.6968398]
District 10: Predicted Outputs: [27.525385   1.1175804  1.0760689  0.7197378]
District 11: Predicted Outputs: [27.073292   1.1081009  1.0809922  0.7239492]
District 12: Predicted Outputs: [29.430374   1.1606772  1.1606772  0.7120706]
District 13: Predicted Outputs: [27.759512   1.1110296  1.111